In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.0-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()




Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Reading package lists

In [2]:
# download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2023-02-24 04:47:02--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar.1’

postgresql-42.2.17. 100%[===================>] 981.19K  1.59MB/s    in 0.6s    

2023-02-24 04:47:04 (1.59 MB/s) - ‘postgresql-42.2.17.jar.1’ saved [1004734/1004734]



In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# start a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview").config("spark.driver.extraClassPat")

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

AttributeError: ignored

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()
     

NameError: ignored

In [7]:
# Retrieve all the rows where the total_votes count is equal to or greater than 20 
vine_votes_df=vine_df.filter("total_votes>=20")
vine_votes_df.show()

NameError: ignored

In [ ]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_ratio_df=vine_votes_df.filter("helpful_votes/total_votes >= 0.5")
votes_ratio_df.show()

In [ ]:
# Retrieves all the rows where a review was written as part of the Vine program (paid)
paid_review_df=votes_ratio_df.filter(votes_ratio_df.vine == "Y")
paid_review_df.show()

In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for paid program
from pyspark.sql.functions import count

# Total nb of reviews 
nb_paid_reviews= paid_review_df.count()
print("The total number of reviews for paid program is:")  
print(nb_paid_reviews)

In [ ]:
# Nnumber of 5 stars reviews for paid program
nb_5star_paid_reviews = paid_review_df.filter(paid_review_df.star_rating == 5).count()
print("The total number of 5-star reviews for paid program is:")  
print(nb_5star_paid_reviews)

In [ ]:
# Percentage of 5-star reviews for paid program
perc_5star_paid = float(nb_5star_paid_reviews)/float(nb_paid_reviews)
print("The percentage of 5-star reviews for paid program is:")  
print(perc_5star_paid)

In [ ]:
# Retrieves all the rows where the review was not part of the Vine program (unpaid)
unpaid_review_df=votes_ratio_df.filter(votes_ratio_df.vine == "N")
unpaid_review_df.show()

In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for unpaid program
# Total nb of reviews 
nb_unpaid_reviews= unpaid_review_df.count()
print("The total number of reviews for unpaid program is:")  
print(nb_unpaid_reviews)

In [ ]:
# Nnumber of 5 stars reviews for unpaid program 
nb_5star_unpaid_reviews = unpaid_review_df.filter(unpaid_review_df.star_rating == 5).count()
print("The total number of 5-star reviews for unpaid program is:")  
print(nb_5star_unpaid_reviews)
     

In [ ]:
# Percentage of 5-star reviews for unpaid program
perc_5star_unpaid = float(nb_5star_unpaid_reviews)/float(nb_unpaid_reviews)
print("The percentage of 5-star reviews for unpaid program is:")  
print(perc_5star_unpaid)